# Titanic Survival Prediction — Logistic Regression
_Generated on 2025-08-29_

This notebook walks through EDA → preprocessing → model training → evaluation → interpretation using Logistic Regression.

**Note:** Place `train.csv` into `../data/` before running.

In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, precision_recall_curve, ConfusionMatrixDisplay
DATA_PATH = Path('../data/train.csv')
df = pd.read_csv(DATA_PATH)
df.head()

## Quick EDA

In [ ]:
df.info(), df.describe(numeric_only=True)

In [ ]:
# Class balance
df['Survived'].value_counts(normalize=True)

In [ ]:
# Simple histograms for key numeric vars
for col in ['Age','Fare']:
    if col in df.columns:
        plt.figure()
        df[col].hist(bins=30)
        plt.title(col)
        plt.xlabel(col)
        plt.ylabel('count')
        plt.show()

## Feature Engineering

In [ ]:
df['FamilySize'] = df['SibSp'].fillna(0) + df['Parch'].fillna(0) + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
df['Title'] = df['Name'].str.extract(r",\s*([^\.]+)\.", expand=False).fillna('Unknown')
rare = df['Title'].value_counts()
rare = rare[rare < 10].index
df['Title'] = df['Title'].replace(rare, 'Rare')
df.head()

## Train/Validation Split

In [ ]:
X = df.drop(columns=['Survived','Cabin','Ticket','Name'])
y = df['Survived']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_valid.shape

## Preprocessing + Logistic Regression Pipeline

In [ ]:
categorical = [c for c in X.columns if c in ['Sex','Embarked','Title']]
numeric = [c for c in X.columns if c in ['Age','Fare','Pclass','SibSp','Parch','FamilySize','IsAlone']]

num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

pre = ColumnTransformer([
    ('num', num_pipe, numeric),
    ('cat', cat_pipe, categorical)
])

clf = LogisticRegression(max_iter=1000, solver='liblinear')
pipe = Pipeline([('pre', pre), ('clf', clf)])
pipe.fit(X_train, y_train)
y_prob = pipe.predict_proba(X_valid)[:,1]
y_pred = (y_prob >= 0.5).astype(int)
print('ROC AUC:', roc_auc_score(y_valid, y_prob))
print(classification_report(y_valid, y_pred, digits=4))

## Curves & Threshold

In [ ]:
fpr, tpr, thr = roc_curve(y_valid, y_prob)
plt.figure()
plt.plot(fpr, tpr, label='ROC')
plt.plot([0,1],[0,1], linestyle='--')
plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC Curve'); plt.legend(); plt.show()

prec, rec, thr2 = precision_recall_curve(y_valid, y_prob)
plt.figure()
plt.plot(rec, prec, label='PR')
plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('Precision-Recall Curve'); plt.legend(); plt.show()

## Coefficients → Odds Ratios

In [ ]:
# Get feature names after preprocessing
ohe = pipe.named_steps['pre'].named_transformers_['cat'].named_steps['ohe']
cat_features = ohe.get_feature_names_out(categorical)
feature_names = list(numeric) + list(cat_features)
coef = pipe.named_steps['clf'].coef_.ravel()
odds = np.exp(coef)
coef_df = pd.DataFrame({'feature': feature_names, 'coef': coef, 'odds_ratio': odds}).sort_values('odds_ratio', ascending=False)
coef_df.head(20)

## Confusion Matrix (0.5 threshold)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_valid, (y_prob>=0.5).astype(int))
plt.title('Confusion Matrix')
plt.show()